# Comparison of Different Machine Learning Algorithm Trained on D-COID Dataset

In [1]:
__author__ = 'yusuf adeshina'
#Institution: Center for Computational Biology University of Kansas, Lawrence
#Project: Machine learning classification can reduce false positives in structure-based virtual screening.
#Scope: To compare different machine learning algorithm trained on our novel dataset D-COID

In [2]:
#Import all necessary libraries
from IPython.display import Image
import matplotlib as mlp
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import discriminant_analysis
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import matthews_corrcoef
import seaborn as sns
from sklearn.utils import shuffle
from configuration
import warnings
#Ignore warnings
warnings.filterwarnings('ignore')

/Users/yusufadeshina/miniconda2/envs/rdkit_env/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/yusufadeshina/miniconda2/envs/rdkit_env/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
#Fixes OpenMP error that kills kernel
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
#Load data
plt.style.use('fivethirtyeight') # Good looking plots
pd.set_option('display.max_columns', None) # Display any number of columns
 
DATA_PATH =cfg.PAPER_FIGURE_DIR+'/final_full_dataset_goldi_like_for_models_new.csv'
 
df = pd.read_csv(DATA_PATH)
df.head()



,PDB,fa_atr,fa_rep,fa_sol,fa_elec,hbond_bb_sc,hbond_sc,Interface_Energy,Total_BSA,Interface_HB,Total_packstats,Interface_unsat,Total_pose_exposed_SASA,interface_hydrophobic_sasa,interface_polar_sasa,6.6,7.6,8.6,16.6,6.7,7.7,8.7,16.7,6.8,7.8,8.8,16.8,6.9,7.9,8.9,16.9,6.15,7.15,8.15,16.15,6.16,7.16,8.16,16.16,6.17,7.17,8.17,16.17,6.35,7.35,8.35,16.35,6.53,7.53,8.53,16.53,SIDE_flex_ALPHA,SIDE_flex_BETA,SIDE_flex_OTHER,BACK_flex_ALPHA,BACK_flex_BETA,BACK_flex_OTHER,TotalElec,TotalHbond,Hphobe_contact,Pi_Pi,T-stack,Cation-pi,Salt_Bridge,diff_entropy,pienergy,fsp3,polarsurfacearea,vdwsa,Label
0,1a28_STR_A_decoys_1,-25.9101,9.64235,3.897390,-0.489556,0.0,0.00000,-17.9156,615.918,0,0.572624,1,0.304661,0.877818,0.122182,2885,639,632,74,489,105,103,12,0,0,0,0,1655,359,386,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32,0,8,10,0,0,-19999.171300,0,8,0,0,0,0,-1.6,65.65,0.25,17.82,352.81,0
1,1a28_STR_A_decoys_2,-28.3938,7.83678,0.865418,0.070411,0.0,0.00000,-23.1729,630.698,0,0.573323,4,0.280210,0.865459,0.134541,3338,735,747,89,0,0,0,0,483,107,113,13,0,0,0,0,0,0,0,0,241,50,50,6,0,0,0,0,212,45,54,6,0,0,0,0,31,0,6,7,0,0,20245.004400,0,15,0,0,0,0,0.0,31.22,0.07,26.30,322.04,0
2,1a28_STR_A_decoys_3,-25.1598,9.36194,-1.626060,-1.294760,0.0,0.00000,-22.9869,655.647,0,0.571481,2,0.281636,0.857508,0.142492,2628,576,599,69,239,57,52,6,242,56,53,6,0,0,0,0,0,0,0,0,480,106,108,13,0,0,0,0,222,47,56,6,0,0,0,0,31,0,2,4,0,0,-53446.046870,0,8,0,0,0,0,0.0,30.05,0.27,20.31,329.90,0
3,1a54_MDC_A_decoys_1,-15.6095,2.82190,-0.927307,-1.841070,0.0,0.00000,-16.8006,522.851,0,0.664623,2,0.598622,0.857328,0.142672,1435,360,442,14,239,71,82,3,402,104,129,5,388,115,125,6,94,25,31,1,65,19,21,1,0,0,0,0,0,0,0,0,0,0,0,0,17,0,0,11,0,0,-103933.696400,0,12,0,0,0,0,-5.2,76.71,0.57,105.76,625.88,0
4,1a54_MDC_A_decoys_2,-23.8063,4.79075,4.132170,-4.171770,0.0,-1.60195,-23.4338,684.668,1,0.695418,3,0.566776,0.707013,0.292987,2490,651,803,27,384,105,123,4,490,134,157,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48,0,0,12,0,0,-898.010755,1,18,0,0,0,0,-6.6,60.81,0.30,127.85,737.65,0


In [5]:
#Shuffle data
df = shuffle(df,random_state=1)
df.head()

,PDB,fa_atr,fa_rep,fa_sol,fa_elec,hbond_bb_sc,hbond_sc,Interface_Energy,Total_BSA,Interface_HB,Total_packstats,Interface_unsat,Total_pose_exposed_SASA,interface_hydrophobic_sasa,interface_polar_sasa,6.6,7.6,8.6,16.6,6.7,7.7,8.7,16.7,6.8,7.8,8.8,16.8,6.9,7.9,8.9,16.9,6.15,7.15,8.15,16.15,6.16,7.16,8.16,16.16,6.17,7.17,8.17,16.17,6.35,7.35,8.35,16.35,6.53,7.53,8.53,16.53,SIDE_flex_ALPHA,SIDE_flex_BETA,SIDE_flex_OTHER,BACK_flex_ALPHA,BACK_flex_BETA,BACK_flex_OTHER,TotalElec,TotalHbond,Hphobe_contact,Pi_Pi,T-stack,Cation-pi,Salt_Bridge,diff_entropy,pienergy,fsp3,polarsurfacearea,vdwsa,Label
1926,4av5_FYZ_A_decoys_4,-18.8276,4.47050,7.71443,-7.96772,0.000000,-5.18113,-20.9068,602.595,4,0.702058,2,0.464334,0.668058,0.331942,1779,477,605,2,0,0,0,0,623,159,200,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48,42,0,0,0,-181729.40730,4,15,0,0,0,0,-5.2,45.33,0.37,99.38,501.23,0
2093,4d9t_0JG_A_decoys_2,-29.2246,7.43535,14.44650,-5.59054,-1.798230,0.00000,-18.1167,736.494,1,0.654778,4,0.307550,0.757354,0.242646,2912,755,848,57,945,258,293,20,378,102,100,7,427,109,108,7,0,0,0,0,131,36,42,3,0,0,0,0,0,0,0,0,0,0,0,0,6,61,18,0,16,16,-75110.08063,2,14,0,0,0,0,-4.0,54.09,0.12,107.95,474.72,0
4881,4ftj_H8K_A_active,-30.7480,8.70259,4.59299,-9.74992,-3.130610,0.00000,-34.0194,680.490,2,0.663436,1,0.436901,0.706453,0.293547,3127,803,1010,16,479,128,154,3,275,65,80,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,57,3,0,37,1,-109011.01020,3,26,0,0,0,0,-3.7,36.33,0.38,74.08,511.90,1
3861,5opr_A3E_A_decoys_3,-25.3625,6.47839,7.63271,-1.36339,-0.547039,0.00000,-16.5562,733.754,1,0.646883,3,0.417743,0.812774,0.187226,3486,897,1025,19,549,151,159,3,455,115,129,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,13,0,1,20,548.00655,1,22,0,0,0,0,-8.2,39.33,0.41,92.35,562.26,0
899,2xyg_TQ8_A_decoys_3,-12.9626,2.62211,4.58836,-2.71313,0.000000,0.00000,-10.1672,509.108,0,0.658581,4,0.499559,0.666572,0.333428,1521,392,449,28,201,50,61,4,255,68,81,4,0,0,0,0,0,0,0,0,166,46,51,4,0,0,0,0,0,0,0,0,0,0,0,0,0,23,6,0,0,10,-61397.40890,0,10,0,0,0,0,-5.4,36.56,0.14,55.13,382.77,0


In [6]:
# Remove the non-numeric columns
df1 = df._get_numeric_data()
numpy_array = df1.as_matrix().astype(np.float)
#68 features 
X =numpy_array[:,0:68]
y=numpy_array[:,68]
y

array([0., 0., 1., ..., 0., 0., 1.])

In [7]:
#Function adapted from https://gist.github.com/jscottcronin/bb7adb5850cdaee4469c
def stratified_cv(X, y, clf_class, random_state=None,shuffle=False, n_folds=10, **kwargs):
    
    stratified_k_fold = cross_validation.StratifiedKFold(y, n_folds=n_folds,shuffle=shuffle,random_state=random_state) 
    y_pred = y.copy()
    for train_index, test_index in stratified_k_fold:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

### All features (Rosetta+NC+LigProp+Szybki+RF+BINANA)

In [16]:
#Accuracy All features (Rosetta+NC+LigProp+Szybki+RF+BINANA)
print('||||||||||||||||||||||||ACCURACY FOR Rosetta+NC+LigProp+Szybki+RF+BINANA|||||||||||||||||||||||||||')
print('XGBoost Classifier:  {:.2f} '.format(metrics.accuracy_score(y, stratified_cv(X, y,XGBClassifier))))
print('Gradient Boosting Classifier:  {:.2f} '.format(metrics.accuracy_score(y, stratified_cv(X, y, ensemble.GradientBoostingClassifier))))
print('Random Forest Classifier:      {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, ensemble.RandomForestClassifier))))
print('Extra Trees Classifier:  {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, ensemble.ExtraTreesClassifier))))
print('Support vector machine(SVM):   {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, svm.SVC))))
print('Linear Discriminant Analysis: {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, discriminant_analysis.LinearDiscriminantAnalysis))))
print('Quadratic Discriminant Analysis: {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, discriminant_analysis.QuadraticDiscriminantAnalysis))))
print('Gaussian Naive Bayes Classifier:  {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, naive_bayes.GaussianNB))))
print('K Nearest Neighbor Classifier: {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, neighbors.KNeighborsClassifier))))



||||||||||||||||||||||||ACCURACY FOR Rosetta+NC+LigProp+Szybki+RF+BINANA|||||||||||||||||||||||||||
XGBoost Classifier:  0.89 
Gradient Boosting Classifier:  0.89 
Random Forest Classifier:      0.86
Extra Trees Classifier:  0.86
Support vector machine(SVM):   0.75
Linear Discriminant Analysis: 0.86
Quadratic Discriminant Analysis: 0.35
Gaussian Naive Bayes Classifier:  0.50
K Nearest Neighbor Classifier: 0.73


In [17]:
#Precision All features (Rosetta+NC+LigProp+Szybki+RF+BINANA)
print('||||||||||||||||||||||||PRECISION FOR Rosetta+NC+LigProp+Szybki+RF+BINANA|||||||||||||||||||||||||||')
print('XGBoost Classifier:  {:.2f} '.format(metrics.precision_score(y, stratified_cv(X, y,XGBClassifier))))
print('Gradient Boosting Classifier:  {:.2f} '.format(metrics.precision_score(y, stratified_cv(X, y, ensemble.GradientBoostingClassifier))))
print('Random Forest Classifier:      {:.2f}'.format(metrics.precision_score(y, stratified_cv(X, y, ensemble.RandomForestClassifier))))
print('Extra Trees Classifier:  {:.2f}'.format(metrics.precision_score(y, stratified_cv(X, y, ensemble.ExtraTreesClassifier))))
print('Support vector machine(SVM):   {:.2f}'.format(metrics.precision_score(y, stratified_cv(X, y, svm.SVC))))
print('Linear Discriminant Analysis: {:.2f}'.format(metrics.precision_score(y, stratified_cv(X, y, discriminant_analysis.LinearDiscriminantAnalysis))))
print('Quadratic Discriminant Analysis: {:.2f}'.format(metrics.precision_score(y, stratified_cv(X, y, discriminant_analysis.QuadraticDiscriminantAnalysis))))
print('Gaussian Naive Bayes Classifier:  {:.2f}'.format(metrics.precision_score(y, stratified_cv(X, y, naive_bayes.GaussianNB))))
print('K Nearest Neighbor Classifier: {:.2f}'.format(metrics.precision_score(y, stratified_cv(X, y, neighbors.KNeighborsClassifier))))


||||||||||||||||||||||||PRECISION FOR Rosetta+NC+LigProp+Szybki+RF+BINANA|||||||||||||||||||||||||||
XGBoost Classifier:  0.85 
Gradient Boosting Classifier:  0.85 
Random Forest Classifier:      0.85
Extra Trees Classifier:  0.85
Support vector machine(SVM):   0.00
Linear Discriminant Analysis: 0.79
Quadratic Discriminant Analysis: 0.28
Gaussian Naive Bayes Classifier:  0.32
K Nearest Neighbor Classifier: 0.45


In [18]:
#Recall All features (Rosetta+NC+LigProp+Szybki+RF+BINANA)
print('||||||||||||||||||||||||RECALL FOR Rosetta+NC+LigProp+Szybki+RF+BINANA|||||||||||||||||||||||||||')
print('XGBoost Classifier:  {:.2f} '.format(metrics.recall_score(y, stratified_cv(X, y,XGBClassifier))))
print('Gradient Boosting Classifier:  {:.2f} '.format(metrics.recall_score(y, stratified_cv(X, y, ensemble.GradientBoostingClassifier))))
print('Random Forest Classifier:      {:.2f}'.format(metrics.recall_score(y, stratified_cv(X, y, ensemble.RandomForestClassifier))))
print('Extra Trees Classifier:  {:.2f}'.format(metrics.recall_score(y, stratified_cv(X, y, ensemble.ExtraTreesClassifier))))
print('Support vector machine(SVM):   {:.2f}'.format(metrics.recall_score(y, stratified_cv(X, y, svm.SVC))))
print('Linear Discriminant Analysis: {:.2f}'.format(metrics.recall_score(y, stratified_cv(X, y, discriminant_analysis.LinearDiscriminantAnalysis))))
print('Quadratic Discriminant Analysis: {:.2f}'.format(metrics.recall_score(y, stratified_cv(X, y, discriminant_analysis.QuadraticDiscriminantAnalysis))))
print('Gaussian Naive Bayes Classifier:  {:.2f}'.format(metrics.recall_score(y, stratified_cv(X, y, naive_bayes.GaussianNB))))
print('K Nearest Neighbor Classifier: {:.2f}'.format(metrics.recall_score(y, stratified_cv(X, y, neighbors.KNeighborsClassifier))))



||||||||||||||||||||||||RECALL FOR Rosetta+NC+LigProp+Szybki+RF+BINANA|||||||||||||||||||||||||||
XGBoost Classifier:  0.67 
Gradient Boosting Classifier:  0.67 
Random Forest Classifier:      0.57
Extra Trees Classifier:  0.55
Support vector machine(SVM):   0.00
Linear Discriminant Analysis: 0.62
Quadratic Discriminant Analysis: 0.98
Gaussian Naive Bayes Classifier:  0.90
K Nearest Neighbor Classifier: 0.25


In [19]:
#ROC AUC All features (Rosetta+NC+LigProp+Szybki+RF+BINANA)
print('||||||||||||||||||||||||ROC AUC FOR Rosetta+NC+LigProp+Szybki+RF+BINANA|||||||||||||||||||||||||||')
print('XGBoost Classifier:  {:.2f} '.format(metrics.roc_auc_score(y, stratified_cv(X, y,XGBClassifier))))
print('Gradient Boosting Classifier:  {:.2f} '.format(metrics.roc_auc_score(y, stratified_cv(X, y, ensemble.GradientBoostingClassifier))))
print('Random Forest Classifier:      {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(X, y, ensemble.RandomForestClassifier))))
print('Extra Trees Classifier:  {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(X, y, ensemble.ExtraTreesClassifier))))
print('Support vector machine(SVM):   {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(X, y, svm.SVC))))
print('Linear Discriminant Analysis: {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(X, y, discriminant_analysis.LinearDiscriminantAnalysis))))
print('Quadratic Discriminant Analysis: {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(X, y, discriminant_analysis.QuadraticDiscriminantAnalysis))))
print('Gaussian Naive Bayes Classifier:  {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(X, y, naive_bayes.GaussianNB))))
print('K Nearest Neighbor Classifier: {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(X, y, neighbors.KNeighborsClassifier))))

||||||||||||||||||||||||ROC AUC FOR Rosetta+NC+LigProp+Szybki+RF+BINANA|||||||||||||||||||||||||||
XGBoost Classifier:  0.81 
Gradient Boosting Classifier:  0.81 
Random Forest Classifier:      0.76
Extra Trees Classifier:  0.76
Support vector machine(SVM):   0.50
Linear Discriminant Analysis: 0.78
Quadratic Discriminant Analysis: 0.56
Gaussian Naive Bayes Classifier:  0.63
K Nearest Neighbor Classifier: 0.57


In [20]:
#Matthew's correlation All features (Rosetta+NC+LigProp+Szybki+RF+BINANA)
print('||||||||||||||||||||||||MATTHEWS CORR COEF FOR Rosetta+NC+LigProp+Szybki+RF+BINANA|||||||||||||||||||||||||||')
print('XGBoost Classifier:  {:.2f} '.format(metrics.matthews_corrcoef(y, stratified_cv(X, y,XGBClassifier))))
print('Gradient Boosting Classifier:  {:.2f} '.format(metrics.matthews_corrcoef(y, stratified_cv(X, y, ensemble.GradientBoostingClassifier))))
print('Random Forest Classifier:      {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(X, y, ensemble.RandomForestClassifier))))
print('Extra Trees Classifier:  {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(X, y, ensemble.ExtraTreesClassifier))))
print('Support vector machine(SVM):   {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(X, y, svm.SVC))))
print('Linear Discriminant Analysis: {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(X, y, discriminant_analysis.LinearDiscriminantAnalysis))))
print('Quadratic Discriminant Analysis: {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(X, y, discriminant_analysis.QuadraticDiscriminantAnalysis))))
print('Gaussian Naive Bayes Classifier:  {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(X, y, naive_bayes.GaussianNB))))
print('K Nearest Neighbor Classifier: {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(X, y, neighbors.KNeighborsClassifier))))

||||||||||||||||||||||||MATTHEWS CORR COEF FOR Rosetta+NC+LigProp+Szybki+RF+BINANA|||||||||||||||||||||||||||
XGBoost Classifier:  0.68 
Gradient Boosting Classifier:  0.68 
Random Forest Classifier:      0.62
Extra Trees Classifier:  0.60
Support vector machine(SVM):   0.00
Linear Discriminant Analysis: 0.62
Quadratic Discriminant Analysis: 0.16
Gaussian Naive Bayes Classifier:  0.26
K Nearest Neighbor Classifier: 0.18


### Different Combinations of Features

In [23]:
#Rosetta component energy term
rosetta =["fa_atr","fa_rep","fa_sol",
          "fa_elec","hbond_bb_sc","hbond_sc"]

#rosetta_plus_nc = Rosetta energy terms plus non-canonical rosetta terms
rosetta_nc=["fa_atr","fa_rep","fa_sol",
            "fa_elec","hbond_bb_sc",
            "hbond_sc","Interface_Energy",
            "Total_BSA","Interface_HB",
            "Total_packstats","Interface_unsat",
            "Total_pose_exposed_SASA",
            "interface_hydrophobic_sasa",
            "interface_polar_sasa"]

#rfscore features 
#(Ref: Ballester PJ, Mitchell JB. \ 
#      A machine learning approach to predicting protein-ligand binding affinity \
#      with applications to molecular docking. Bioinformatics. 2010; 26:1169-75)
rf = ["6.6","7.6","8.6","16.6","6.7","7.7","8.7",
      "16.7","6.8","7.8","8.8","16.8","6.9","7.9",
      "8.9","16.9","6.15","7.15","8.15","16.15","6.16",
      "7.16","8.16","16.16","6.17","7.17","8.17",
      "16.17","6.35","7.35","8.35","16.35","6.53",
      "7.53","8.53","16.53"]

#binana features
# Ref: Durrant JD, McCammon JA. \
#.     BINANA: A Novel Algorithm for Ligand-Binding Characterization.\ 
#.     J Mol Graph Model. 2011; 29:888-93.)
binana= ["SIDE_flex_ALPHA","SIDE_flex_BETA",
         "SIDE_flex_OTHER","BACK_flex_ALPHA",
         "BACK_flex_BETA","BACK_flex_OTHER",
         "TotalElec","TotalHbond","Hphobe_contact",
         "Pi_Pi","T-stack","Cation-pi","Salt_Bridge"]

#All Rosetta features and ChemAxon Ligand Features
rosetta_nc_ligprop = ["fa_atr","fa_rep","fa_sol",
                      "fa_elec","hbond_bb_sc","hbond_sc",
                      "Interface_Energy","Total_BSA",
                      "Interface_HB","Total_packstats",
                      "Interface_unsat","Total_pose_exposed_SASA",
                      "interface_hydrophobic_sasa",
                      "interface_polar_sasa","pienergy",
                      "fsp3","polarsurfacearea","vdwsa"]

#All Rosetta features, ChemAxon Ligand Features and OpenEye Szybki Entropy
rosetta_nc_ligprop_szybki = ["fa_atr","fa_rep","fa_sol","fa_elec",
                             "hbond_bb_sc","hbond_sc",
                             "Interface_Energy", "Total_BSA",
                             "Interface_HB","Total_packstats", 
                             "Interface_unsat","Total_pose_exposed_SASA",
                             "interface_hydrophobic_sasa",
                             "interface_polar_sasa","pienergy","fsp3",
                             "polarsurfacearea","vdwsa","diff_entropy"]

#All Rosetta features, ChemAxon Ligand Features, OpenEye Szybki Entropy and rfscore features
rosetta_nc_ligprop_szybki_rf = ["fa_atr","fa_rep","fa_sol","fa_elec",
                                "hbond_bb_sc","hbond_sc","Interface_Energy", 
                                "Total_BSA","Interface_HB","Total_packstats",
                                "Interface_unsat","Total_pose_exposed_SASA", 
                                "interface_hydrophobic_sasa","interface_polar_sasa",
                                "pienergy","fsp3","polarsurfacearea","vdwsa",
                                "diff_entropy","6.6","7.6","8.6","16.6","6.7","7.7",
                                "8.7","16.7","6.8","7.8","8.8","16.8","6.9","7.9",
                                "8.9","16.9","6.15","7.15","8.15","16.15","6.16",
                                "7.16","8.16","16.16","6.17","7.17","8.17","16.17",
                                "6.35","7.35","8.35","16.35","6.53","7.53","8.53","16.53"]

#All Rosetta features, ChemAxon Ligand Features, OpenEye Szybki Entropy and binana features
rosetta_nc_ligprop_szybki_binana = ["fa_atr","fa_rep","fa_sol","fa_elec","hbond_bb_sc",
                                    "hbond_sc","Interface_Energy","Total_BSA","Interface_HB",
                                    "Total_packstats","Interface_unsat","Total_pose_exposed_SASA",
                                    "interface_hydrophobic_sasa","interface_polar_sasa","pienergy",
                                    "fsp3","polarsurfacearea","vdwsa","diff_entropy","SIDE_flex_ALPHA",
                                    "SIDE_flex_BETA","SIDE_flex_OTHER","BACK_flex_ALPHA","BACK_flex_BETA",
                                    "BACK_flex_OTHER","TotalElec","TotalHbond","Hphobe_contact","Pi_Pi",
                                    "T-stack","Cation-pi","Salt_Bridge"]

#### EEFECT OF FEATURE REMOVAL WITH XGBOOST

In [35]:
#Populate DataFrames with the different features
df_rosetta = df[rosetta]
df_rosetta_nc = df[rosetta_nc]
df_rf = df[rf]
df_binana = df[binana]
df_rosetta_nc_ligprop = df[rosetta_nc_ligprop]
df_rosetta_nc_ligprop_szybki = df[rosetta_nc_ligprop_szybki]
df_rosetta_nc_ligprop_szybki_rf = df[rosetta_nc_ligprop_szybki_rf]
df_rosetta_nc_ligprop_szybki_binana = df[rosetta_nc_ligprop_szybki_binana]
df_rosetta_nc_ligprop_szybki_rf_binana = df

In [45]:
#Remove Non-numeric data
df_rosetta1 = df_rosetta._get_numeric_data()
df_rosetta_nc1 = df_rosetta_nc._get_numeric_data()
df_rf1 = df_rf._get_numeric_data()
df_binana1 = df_binana._get_numeric_data()
df_rosetta_nc_ligprop1 = df_rosetta_nc_ligprop._get_numeric_data()
df_rosetta_nc_ligprop_szybki1 = df_rosetta_nc_ligprop_szybki._get_numeric_data()
df_rosetta_nc_ligprop_szybki_rf1 = df_rosetta_nc_ligprop_szybki_rf._get_numeric_data()
df_rosetta_nc_ligprop_szybki_binana1 = df_rosetta_nc_ligprop_szybki_binana._get_numeric_data()
df_rosetta_nc_ligprop_szybki_rf_binana1 = df_rosetta_nc_ligprop_szybki_rf_binana._get_numeric_data()
#Convert to numpy array
numpy_array_rosetta = df_rosetta1.as_matrix().astype(np.float)
numpy_array_rosetta_nc = df_rosetta_nc1.as_matrix().astype(np.float)
numpy_array_rf = df_rf1.as_matrix().astype(np.float)
numpy_array_binana = df_binana1.as_matrix().astype(np.float)
numpy_array_rosetta_nc_ligprop = df_rosetta_nc_ligprop1.as_matrix().astype(np.float)
numpy_array_rosetta_nc_ligprop_szybki = df_rosetta_nc_ligprop_szybki1.as_matrix().astype(np.float)
numpy_array_rosetta_nc_ligprop_szybki_rf = df_rosetta_nc_ligprop_szybki_rf1.as_matrix().astype(np.float)
numpy_array_rosetta_nc_ligprop_szybki_binana = df_rosetta_nc_ligprop_szybki_binana1.as_matrix().astype(np.float)
numpy_array_rosetta_nc_ligprop_szybki_rf_binana = df_rosetta_nc_ligprop_szybki_rf_binana1.as_matrix().astype(np.float)[:,:-1] 
#X_rosetta =numpy_array[:,:]
#np.shape(X_rosetta)

In [27]:
numpy_array_rosetta_nc.shape

(5493, 14)

In [46]:
#Accuracy of XGBoost with different combinations of features
print('||||||||||||||||||||||||ACCURACY(XGBOOST WITH DIFFERENT FEATURES)|||||||||||||||||||||||||||')
print('Rosetta (Reweighted):   {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(numpy_array_rosetta, y,XGBClassifier))))
print('RF (Reweighted):   {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(numpy_array_rf, y,XGBClassifier))))
print('BINANA (Reweighted):   {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(numpy_array_binana, y,XGBClassifier))))
print('Rosetta+NC (Reweighted):   {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(numpy_array_rosetta_nc, y,XGBClassifier))))
print('Rosetta+NC+LigProp (Reweighted):   {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki (Reweighted):   {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+RF (Reweighted):   {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_rf, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+BINANA (Reweighted):   {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_binana, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+RF+BINANA (Reweighted):   {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_rf_binana, y,XGBClassifier))))

||||||||||||||||||||||||ACCURACY(XGBOOST WITH DIFFERENT FEATURES)|||||||||||||||||||||||||||
Rosetta (Reweighted):   0.85
RF (Reweighted):   0.78
BINANA (Reweighted):   0.83
Rosetta+NC (Reweighted):   0.86
Rosetta+NC+LigProp (Reweighted):   0.86
Rosetta+NC+LigProp+Szybki (Reweighted):   0.86
Rosetta+NC+LigProp+Szybki+RF (Reweighted):   0.88
Rosetta+NC+LigProp+Szybki+BINANA (Reweighted):   0.87
Rosetta+NC+LigProp+Szybki+RF+BINANA (Reweighted):   0.89


In [47]:
#Precison of XGBoost with different combinations of features
print('||||||||||||||||||||||||PRECISION(XGBOOST WITH DIFFERENT FEATURES)|||||||||||||||||||||||||||')
print('Rosetta (Reweighted):   {:.2f}'.format(metrics.precision_score(y, stratified_cv(numpy_array_rosetta, y,XGBClassifier))))
print('RF (Reweighted):   {:.2f}'.format(metrics.precision_score(y, stratified_cv(numpy_array_rf, y,XGBClassifier))))
print('BINANA (Reweighted):   {:.2f}'.format(metrics.precision_score(y, stratified_cv(numpy_array_binana, y,XGBClassifier))))
print('Rosetta+NC (Reweighted):   {:.2f}'.format(metrics.precision_score(y, stratified_cv(numpy_array_rosetta_nc, y,XGBClassifier))))
print('Rosetta+NC+LigProp (Reweighted):   {:.2f}'.format(metrics.precision_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki (Reweighted):   {:.2f}'.format(metrics.precision_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+RF (Reweighted):   {:.2f}'.format(metrics.precision_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_rf, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+BINANA (Reweighted):   {:.2f}'.format(metrics.precision_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_binana, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+RF+BINANA (Reweighted):   {:.2f}'.format(metrics.precision_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_rf_binana, y,XGBClassifier))))

||||||||||||||||||||||||PRECISION(XGBOOST WITH DIFFERENT FEATURES)|||||||||||||||||||||||||||
Rosetta (Reweighted):   0.75
RF (Reweighted):   0.66
BINANA (Reweighted):   0.75
Rosetta+NC (Reweighted):   0.79
Rosetta+NC+LigProp (Reweighted):   0.79
Rosetta+NC+LigProp+Szybki (Reweighted):   0.79
Rosetta+NC+LigProp+Szybki+RF (Reweighted):   0.83
Rosetta+NC+LigProp+Szybki+BINANA (Reweighted):   0.82
Rosetta+NC+LigProp+Szybki+RF+BINANA (Reweighted):   0.85


In [48]:
#Recall of XGBoost with different combinations of features
print('||||||||||||||||||||||||RECALL(XGBOOST WITH DIFFERENT FEATURES)|||||||||||||||||||||||||||')
print('Rosetta (Reweighted):   {:.2f}'.format(metrics.recall_score(y, stratified_cv(numpy_array_rosetta, y,XGBClassifier))))
print('RF (Reweighted):   {:.2f}'.format(metrics.recall_score(y, stratified_cv(numpy_array_rf, y,XGBClassifier))))
print('BINANA (Reweighted):   {:.2f}'.format(metrics.recall_score(y, stratified_cv(numpy_array_binana, y,XGBClassifier))))
print('Rosetta+NC (Reweighted):   {:.2f}'.format(metrics.recall_score(y, stratified_cv(numpy_array_rosetta_nc, y,XGBClassifier))))
print('Rosetta+NC+LigProp (Reweighted):   {:.2f}'.format(metrics.recall_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki (Reweighted):   {:.2f}'.format(metrics.recall_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+RF (Reweighted):   {:.2f}'.format(metrics.recall_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_rf, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+BINANA (Reweighted):   {:.2f}'.format(metrics.recall_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_binana, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+RF+BINANA (Reweighted):   {:.2f}'.format(metrics.recall_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_rf_binana, y,XGBClassifier))))

||||||||||||||||||||||||RECALL(XGBOOST WITH DIFFERENT FEATURES)|||||||||||||||||||||||||||
Rosetta (Reweighted):   0.57
RF (Reweighted):   0.30
BINANA (Reweighted):   0.50
Rosetta+NC (Reweighted):   0.60
Rosetta+NC+LigProp (Reweighted):   0.60
Rosetta+NC+LigProp+Szybki (Reweighted):   0.60
Rosetta+NC+LigProp+Szybki+RF (Reweighted):   0.65
Rosetta+NC+LigProp+Szybki+BINANA (Reweighted):   0.62
Rosetta+NC+LigProp+Szybki+RF+BINANA (Reweighted):   0.67


In [49]:
#ROC AUC of XGBoost with different combinations of features
print('||||||||||||||||||||||||ROC AUC(XGBOOST WITH DIFFERENT FEATURES)|||||||||||||||||||||||||||')
print('Rosetta (Reweighted):   {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(numpy_array_rosetta, y,XGBClassifier))))
print('RF (Reweighted):   {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(numpy_array_rf, y,XGBClassifier))))
print('BINANA (Reweighted):   {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(numpy_array_binana, y,XGBClassifier))))
print('Rosetta+NC (Reweighted):   {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(numpy_array_rosetta_nc, y,XGBClassifier))))
print('Rosetta+NC+LigProp (Reweighted):   {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki (Reweighted):   {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+RF (Reweighted):   {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_rf, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+BINANA (Reweighted):   {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_binana, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+RF+BINANA (Reweighted):   {:.2f}'.format(metrics.roc_auc_score(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_rf_binana, y,XGBClassifier))))

||||||||||||||||||||||||ROC AUC(XGBOOST WITH DIFFERENT FEATURES)|||||||||||||||||||||||||||
Rosetta (Reweighted):   0.76
RF (Reweighted):   0.62
BINANA (Reweighted):   0.72
Rosetta+NC (Reweighted):   0.77
Rosetta+NC+LigProp (Reweighted):   0.77
Rosetta+NC+LigProp+Szybki (Reweighted):   0.77
Rosetta+NC+LigProp+Szybki+RF (Reweighted):   0.80
Rosetta+NC+LigProp+Szybki+BINANA (Reweighted):   0.79
Rosetta+NC+LigProp+Szybki+RF+BINANA (Reweighted):   0.81


In [50]:
#Matthews correlation coefficient of XGBoost with different combinations of features
print('||||||||||||||||||||||||MAT. CORR. COEFF.(XGBOOST WITH DIFFERENT FEATURES)|||||||||||||||||||||||||||')
print('Rosetta (Reweighted):   {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(numpy_array_rosetta, y,XGBClassifier))))
print('RF (Reweighted):   {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(numpy_array_rf, y,XGBClassifier))))
print('BINANA (Reweighted):   {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(numpy_array_binana, y,XGBClassifier))))
print('Rosetta+NC (Reweighted):   {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(numpy_array_rosetta_nc, y,XGBClassifier))))
print('Rosetta+NC+LigProp (Reweighted):   {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(numpy_array_rosetta_nc_ligprop, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki (Reweighted):   {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+RF (Reweighted):   {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_rf, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+BINANA (Reweighted):   {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_binana, y,XGBClassifier))))
print('Rosetta+NC+LigProp+Szybki+RF+BINANA (Reweighted):   {:.2f}'.format(metrics.matthews_corrcoef(y, stratified_cv(numpy_array_rosetta_nc_ligprop_szybki_rf_binana, y,XGBClassifier))))

||||||||||||||||||||||||MAT. CORR. COEFF.(XGBOOST WITH DIFFERENT FEATURES)|||||||||||||||||||||||||||
Rosetta (Reweighted):   0.56
RF (Reweighted):   0.34
BINANA (Reweighted):   0.51
Rosetta+NC (Reweighted):   0.60
Rosetta+NC+LigProp (Reweighted):   0.60
Rosetta+NC+LigProp+Szybki (Reweighted):   0.60
Rosetta+NC+LigProp+Szybki+RF (Reweighted):   0.66
Rosetta+NC+LigProp+Szybki+BINANA (Reweighted):   0.64
Rosetta+NC+LigProp+Szybki+RF+BINANA (Reweighted):   0.68
